In [0]:
import tensorflow as tf

In [2]:
!pip install pandas-datareader

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [4]:
tf.__version__

'2.2.0'

In [0]:
class sAI_Trader():
  
  def __init__(self, state_size, action_space = 3, model_name = "sAITrader"):
     #Actions are stay, buy and sell
     self.state_size = state_size
     self.action_space = action_space
     self.model_name = model_name
     self.inventory = [] #To store all the stocks bought, to sell only these
     self.memory = deque(maxlen = 2000) #This is for experience replay

     self.gamma = 0.95 #For immediate vs future winnings (reward cashing in)
     self.epsilon = 1.0 #This is exploration vs exploitation - to keep exploring all new states
     self.epsilon_final = 0.01
     self.epsilon_decay = 0.995 
     self.model = self.model_builder()
    
  def model_builder(self):

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = 'relu', input_dim = self.state_size)) 
    model.add(tf.keras.layers.Dense(units = 32, activation = 'relu')) 
    model.add(tf.keras.layers.Dense(units = 32, activation = 'relu')) 

    model.add(tf.keras.layers.Dense(units = self.action_space, activation = 'linear')) #Linear activation is used here as the output is changed based on reward which is a real number and not a class

    model.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model
  
  def trade(self, state):

    if random.random() <= self.epsilon:  #First, it is always less than epsilon, and at first we explore
      return random.randrange(self.action_space)

    else:
      actions = self.model.predict(state) #Here, we exploit
      return np.argmax(actions[0]) #Due to the shape of the output   

  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [0]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

In [0]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])  #prepending to cover up the negative timesteps
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i])) #our data is the difference of stock prices, scaled down
    
  return np.array([state])

In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [0]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1 #Last one is the prediction

In [0]:
trader = sAI_Trader(window_size)

In [13]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [0]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 126.919998
AI Trader bought:  $ 127.599998
AI Trader bought:  $ 127.300003
AI Trader bought:  $ 127.879997
AI Trader bought:  $ 127.029999
AI Trader bought:  $ 128.110001
AI Trader bought:  $ 127.500000
AI Trader bought:  $ 124.529999
AI Trader bought:  $ 125.430000
AI Trader sold:  $ 126.440002  Profit: - $ 0.479996
AI Trader sold:  $ 120.070000  Profit: - $ 7.529999
AI Trader bought:  $ 123.279999
AI Trader sold:  $ 125.660004  Profit: - $ 1.639999
AI Trader sold:  $ 125.610001  Profit: - $ 2.269997
AI Trader sold:  $ 126.820000  Profit: - $ 0.209999
AI Trader bought:  $ 128.509995
AI Trader sold:  $ 129.619995  Profit: $ 1.509995
AI Trader bought:  $ 132.070007
AI Trader sold:  $ 130.750000  Profit: $ 3.250000
AI Trader bought:  $ 125.220001
AI Trader bought:  $ 124.500000
AI Trader bought:  $ 122.370003


  3%|▎         | 33/1258 [00:05<03:07,  6.53it/s]

AI Trader bought:  $ 121.300003


  3%|▎         | 34/1258 [00:07<18:59,  1.07it/s]

...

  9%|▉         | 115/1258 [05:26<54:25,  2.86s/it]

AI Trader bought:  $ 118.029999


  9%|▉         | 116/1258 [05:29<53:56,  2.83s/it]

AI Trader bought:  $ 117.809998


  9%|▉         | 117/1258 [05:32<53:46,  2.83s/it]